<a href="https://colab.research.google.com/github/anhnguyen17/JAPAN2020/blob/main/FinalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data

In [ ]:
#Tweet/Short text dataset


tweet = [("Bitcoin","Sign up with this website to receive TEN DOLLARS in Bitcoin! First time users only"),
         ("Marvel","Marvel Mobile, play the official Spider-man game on ur mobile web right now."),
         ("Police","Exclusive: Watch how George Floyed was killed in police custody on this link"),
         ("iPhone","Visit our website for a BIG DISCOUNT on your iPhone purchase!"),
         ("Walmart","Lucky shoppers! You've won a THOUSAND DOLLAR Walmart gift card! Go to this link to claim now."),
         ("soccer", "World Cup 2022 is here! Check out the game schedule on this link RIGHT NOW..."), 
         ("travel", "2 hours left to join our site. Your complimentary FOUR STAR Ibiza Holiday or ONE THOUSAND cash await collection"),                       
         ("dating","Talk sexy!! Fall in love in the worlds most discreet text dating service. Sign up at this link and see who you could meet.")
         
         #("Spanish", "Learn Spanish with natives. Log onto this link now"),
         #("Records","Congratulations ur awarded either a year supply of CDs from Virgin Records when subscribing to our channel"),
         #("donations","Urgent!!!! Please we still need your help! Donations are still needed for Alicia surgery.. "),
         #("tennis","Get free tennis lessons by clicking on this link!"),
         ]

test_tweet = [("Spanish", "Learn Spanish with natives. Log onto this link now"),
         ("Records","Congratulations ur awarded either a year supply of CDs from Virgin Records when subscribing to our channel"),
         ("donations","Urgent Alicia needs surgery now!! Donations accepted on this website.. "),
         ("tennis","Get free tennis lessons by clicking on this link!")]





#Save / Load Model

In [6]:

from google.colab import drive
drive.mount('/content/gdrive')

def save():
  path = "/content/gdrive/MyDrive/Colab Notebooks/T5_models" 
  sys.path.append(os.path.abspath(path))

  tokenizer.save_pretrained(path)
  t5_model.save_pretrained(path)


Mounted at /content/gdrive


# T5 Fine tuning

In [2]:

###Installing transformer
print('Install transformers')

!pip install transformers==2.9.0

###Import all needed package
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import sys
import os

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

Install transformers
     |████████████████████████████████| 645kB 5.3MB/s 
     |████████████████████████████████| 5.6MB 10.7MB/s 
     |████████████████████████████████| 1.2MB 38.1MB/s 
     |████████████████████████████████| 901kB 53.2MB/s 


In [3]:

#eliminate randomness for reproduction purposes
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

def get_model(model_name):
  tokenizer = T5Tokenizer.from_pretrained(model_name)
  t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

  return tokenizer, t5_model
def get_optimizer(model, epsilon = 1e-8, learning_rate = 1e-4 ):
  no_decay = ["bias", "LayerNorm.weight"]
  optimizer_grouped_parameters = [
      {
          "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
          "weight_decay": 0.0,
      },
      {
          "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
          "weight_decay": 0.0,
      },
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4, eps=1e-8)
  return optimizer
def graph_training_loss(loss_data):
  steps = [i for i in range(len(loss_data))]
  plt.plot(steps, loss_data)
  plt.title('Loss')
  plt.xlabel('Steps')
  plt.ylabel('Loss')
  plt.show()

In [25]:

def train(data, tokenizer, t5_model,epochs = 30):
  optimizer = get_optimizer(t5_model)
  loss_data = []
  for epoch in range(epochs):
    print ("epoch ",epoch)
    running_loss = 0
    for input,output in data:
      input_kw = "tweet: "+input+ " </s>"
      output_tweet = output+" </s>"

      tokenized_input = tokenizer.encode_plus(input_kw,  max_length=100, pad_to_max_length=True,return_tensors="pt")
      tokenized_output = tokenizer.encode_plus(output_tweet, max_length=100, pad_to_max_length=True,return_tensors="pt")

      # the forward function that creates the correct decoder_input_ids
      output = t5_model(input_ids=tokenized_input["input_ids"], 
                        lm_labels=tokenized_output["input_ids"],
                        decoder_attention_mask=tokenized_output["attention_mask"],
                        attention_mask=tokenized_input["attention_mask"])
      loss = output[0]
      logits = output[1]
      running_loss += loss.item() 

      #calculate gradient
      loss.backward()

      #update the parameters
      optimizer.step()

      #clear gradients of all Var
      optimizer.zero_grad()
    loss_data.append(running_loss)
    print(running_loss)

  graph_training_loss(loss_data)

  return (tokenizer,t5_model)

def beam_decoder(keyword, t5_model, tokenizer):

  test_kw = 'tweet: ' + keyword  + '</s>'
  test_tokenized = tokenizer.encode_plus(test_kw, return_tensors="pt")
  beam_outputs = t5_model.generate(
    input_ids = test_tokenized["input_ids"], 
    attention_mask=test_tokenized["attention_mask"],
    max_length=64, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=1, 
    early_stopping=True)

  print("Output for Beam decoder:\n" + 100 * '-')
  # now we have 2 output sequences
  for i, sample_output in enumerate(beam_outputs):
    print(tokenizer.decode(sample_output, skip_special_tokens=True))

def sample_decoder(keyword , t5_model, tokenizer):
  test_kw = 'tweet: ' + keyword +'</s>'
  test_tokenized = tokenizer.encode_plus(test_kw, return_tensors="pt")

  # use temperature to decrease the sensitivity to low probability candidates
  sample_output = t5_model.generate(
      input_ids = test_tokenized["input_ids"], 
      attention_mask=test_tokenized["attention_mask"],
      do_sample=True, 
      max_length=64, 
      top_k=0, 
      temperature=0.7
  )

  print("Output for Sample Decoder:\n" + 100 * '-')
  print(tokenizer.decode(sample_output[0], skip_special_tokens=True))
  
  
def topPtopK_decoder(keyword , t5_model, tokenizer):
  res = []
  test_kw = 'tweet: ' + keyword +'</s>'
  test_tokenized = tokenizer.encode_plus(test_kw, return_tensors="pt")


  sample_outputs = t5_model.generate(
    input_ids = test_tokenized["input_ids"], 
    attention_mask=test_tokenized["attention_mask"],
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3)

  print("Output for Top P top K decoder:\n" + 100 * '-')
  for i, sample_output in enumerate(sample_outputs):
    print(tokenizer.decode(sample_output, skip_special_tokens=True))


In [ ]:
###Code to train tweet dataset

tokenizer , t5_model = get_model('t5-base')

###Replace tweet data
tokenizer, t5_model = train(tweet, tokenizer, t5_model,epochs = 20)

save()

#Model evaluation

In [18]:

def print_output(func , model ,tokenizer):
  reference = []

  for input,output in test_tweet:
    reference.append(output)
    print("Keyword: "+ input)
    func(input, model ,tokenizer)

def print_output(keyword, model ,tokenizer):
  print("Keyword: "+ keyword)
  beam_decoder(keyword, model ,tokenizer)
  print()
  sample_decoder(keyword, model ,tokenizer)
  print()
  topPtopK_decoder(keyword, model ,tokenizer)
  print()




In [7]:
path = "/content/gdrive/MyDrive/Colab Notebooks/T5_models" 
sys.path.append(os.path.abspath(path))
tokenizer= T5Tokenizer.from_pretrained(path)
t5_model = T5ForConditionalGeneration.from_pretrained(path)

In [27]:
#Filled in your desired keywords for message generating
print_output('showbiz', t5_model, tokenizer)
print()


Keyword: showbiz
Output for Beam decoder:
----------------------------------------------------------------------------------------------------
Exclusive: showbizbiz.com has a new look at what's going on in the worlds of television and the arts - and this is the first time we've seen it!

Output for Sample Decoder:
----------------------------------------------------------------------------------------------------
Showbizbizbizbiznews - Join the conversation! Join the conversation!<extra_id_0>bizbiz.com. This entry was posted in showbiz, showbiz news, showbiz news, showbiz culture, showbiz + 1 other person to discuss this

Output for Top P top K decoder:
----------------------------------------------------------------------------------------------------
Sign up with this website to receive TEN DOLLARS in the low 48 hours of sunday, september 20. Also check out our other blog posts: showbizbiz, get your own account and start creating memories on this
Showbizbizbiza is on the fritz! Follo